# AlphabetNet - Entrenamiento en Google Colab

**Tarea**: Regex → Alfabeto del Autómata

Este notebook entrena AlphabetNet para predecir el alfabeto (símbolos A-L) que pertenecen a un autómata dado su expresión regular.

## Métricas objetivo:
- F1 macro ≥ 0.92
- F1 mínimo ≥ 0.85
- Exactitud de conjunto ≥ 0.90
- ECE ≤ 0.05


## 1. Configuración inicial


In [ ]:
# Clonar repositorio
!git clone https://github.com/xLuisPc/ModelosLenguajes.git
%cd ModelosLenguajes

# Instalar dependencias
%pip install -r requirements.txt -q

print("✓ Repositorio clonado y dependencias instaladas")


In [ ]:
# Verificar GPU
import torch
import os
from pathlib import Path

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠ Se usará CPU (más lento)")


## 2. Preparar dataset


In [ ]:
# Verificar si el dataset ya existe en el repo
dataset_path = Path('data/dataset_regex_sigma.csv')

if dataset_path.exists():
    import pandas as pd
    df = pd.read_csv(dataset_path)
    print(f"✓ Dataset encontrado: {len(df)} autómatas")
    print(f"  Columnas: {list(df.columns)}")
    print(f"\n  Primeras 3 filas:")
    print(df.head(3))
else:
    print("✗ Dataset no encontrado")
    print("\nOpciones:")
    print("  1. Generar desde dataset3000.csv (si está en el repo):")
    print("     !python scripts/create_regex_sigma_dataset.py")
    print("\n  2. Subir dataset_regex_sigma.csv manualmente:")
    print("     from google.colab import files")
    print("     files.upload()")


In [ ]:
# Si dataset3000.csv existe pero dataset_regex_sigma.csv no, generar dataset
if Path('dataset3000.csv').exists() and not dataset_path.exists():
    print("Generando dataset_regex_sigma.csv...")
    !python scripts/create_regex_sigma_dataset.py
    
    if dataset_path.exists():
        print("✓ Dataset generado exitosamente")
    else:
        print("✗ Error al generar dataset")
else:
    print("Dataset ya existe o dataset3000.csv no encontrado")


## 3. Entrenamiento


In [ ]:
# Entrenar modelo
!python train.py \
  --train_data data/dataset_regex_sigma.csv \
  --val_data data/dataset_regex_sigma.csv \
  --checkpoint_dir checkpoints \
  --use_scheduler


## 4. Búsqueda de umbrales óptimos


In [ ]:
# Buscar umbrales óptimos por símbolo
!python find_thresholds.py \
  --checkpoint checkpoints/best.pt \
  --val_data data/dataset_regex_sigma.csv \
  --output_dir checkpoints


## 5. Inferencia - Demo


In [ ]:
# Ejemplo de inferencia: predecir alfabeto desde regex
!python infer.py \
  --checkpoint checkpoints/best.pt \
  --regex "(AB)*C" \
  --thresholds checkpoints/thresholds.json


## 6. Exportar modelo


In [ ]:
# Exportar modelo a ONNX con thresholds
!python export_model.py \
  --checkpoint checkpoints/best.pt \
  --thresholds checkpoints/thresholds.json \
  --enhanced_checkpoint checkpoints/best_with_thresholds.pt \
  --output alphabetnet.onnx


## 7. Descargar resultados


In [ ]:
# Descargar archivos importantes
from google.colab import files

files_to_download = [
    'checkpoints/best.pt',
    'checkpoints/best_with_thresholds.pt',
    'checkpoints/thresholds.json',
    'checkpoints/train_log.csv',
    'checkpoints/threshold_eval.csv',
    'alphabetnet.onnx'
]

print("Descargando archivos...")
for filepath in files_to_download:
    path = Path(filepath)
    if path.exists():
        try:
            files.download(str(path))
            print(f"✓ {filepath} descargado")
        except Exception as e:
            print(f"✗ Error al descargar {filepath}: {e}")
    else:
        print(f"⚠ {filepath} no existe")

print("\n✓ Descarga completada")
